In [3]:
import pandas as pd
import numpy as np
import os
# encoding = "ISO-8859-1",

weatherData = pd.read_csv("Data/cleaned_weatherdata_withflagfinal.csv")

weatherData['date'] = pd.to_datetime(weatherData['date'])

for i in range (5,6):
    fullData = None
    combinedData = None
    combinedDataForDrivers = None
    if(i<10):
        fn = "0" + str(i)
    else:
        fn = str(i)
        
                
    for taxiData in pd.read_csv("Data/combinedData_yellow_tripdata_2016-" + fn + ".csv", sep=",", chunksize = 200000, iterator = True):            
    #     Dataset for driver interface
        df = pd.DataFrame() # create new df
        key_columns = ['Pickup_Date', 'Hour_of_day', 'pickup_latitude_2decimal', 'pickup_longitude_2decimal']
        df = taxiData.groupby(key_columns, as_index=False)["fare_amount"].count()
        #Rename Fare Amount column
        df = df.rename(columns={'fare_amount': 'count_of_pickups'})
        
        try:
            fullData = pd.concat([fullData, df])
        except:
            fullData = df
#         x = x + 1
#         if (x == 3):
#             break
        
    fullData['Pickup_Date'] = pd.to_datetime(fullData['Pickup_Date'])
    testcombinedDataForDrivers = pd.merge(fullData, weatherData,  how='inner', left_on=['Pickup_Date','Hour_of_day'], right_on = ['date','hour']) 
    testcombinedDataForDrivers.to_csv("Data/testcombinedForDriver_yellow_tripdata_2016-" + fn + ".csv", sep=',', index = False)
#         grouping again 
    fullData = fullData.groupby(key_columns, as_index=False)["count_of_pickups"].sum()
    fullData['Pickup_Date'] = pd.to_datetime(fullData['Pickup_Date'])
    fullData['Day_of_week'] = fullData['Pickup_Date'].dt.dayofweek
    fullData["Workday_flag"]=1
    fullData.loc[((fullData['Pickup_Date']=='2016-01-01')|
                      (fullData['Pickup_Date']=='2016-01-18')|
                      (fullData['Pickup_Date']=='2016-02-12')|
                      (fullData['Pickup_Date']=='2016-02-15')|
                      (fullData['Pickup_Date']=='2016-05-30')|
                      (fullData['Pickup_Date']=='2016-07-04')|
                      (fullData['Pickup_Date']=='2016-09-15')|
                      (fullData['Pickup_Date']=='2016-10-10')|
                      (fullData['Pickup_Date']=='2016-11-08')|
                      (fullData['Pickup_Date']=='2016-11-11')|
                      (fullData['Pickup_Date']=='2016-11-24')|
                      (fullData['Pickup_Date']=='2016-12-25')|
                      (fullData['Day_of_week']==5)|
                        (fullData['Day_of_week']==6)),['Workday_flag']]=0
    print("Shape of new df = " + str(fullData.shape) )
    print("columns in new df : " + str(list(fullData.columns.values)))    
        
    #     Combine weather data
    fullData['Pickup_Date'] = pd.to_datetime(fullData['Pickup_Date'])
    weatherData['date'] = pd.to_datetime(weatherData['date'])
    combinedDataForDrivers = pd.merge(fullData, weatherData,  how='inner', left_on=['Pickup_Date','Hour_of_day'], right_on = ['date','hour'])
    #drop NAs
    combinedDataForDrivers.dropna(how='any')
    print("Shape of combinedDataForDrivers = " + str(combinedDataForDrivers.shape) )
    combinedDataForDrivers.to_csv("Data/combinedForDriver_yellow_tripdata_2016-" + fn + ".csv", sep=',', index = False)

Shape of new df = (121957, 7)
columns in new df : ['Pickup_Date', 'Hour_of_day', 'pickup_latitude_2decimal', 'pickup_longitude_2decimal', 'count_of_pickups', 'Day_of_week', 'Workday_flag']
Shape of combinedDataForDrivers = (153029, 20)
